# Milestone 1

### Data

In [1]:
import numpy as np
X = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
y = np.array([[0],[1],[1],[0]], dtype=np.float32)

### Build net with your library

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

# 1. can we even *see* lib?
import os
# print('lib contents:', os.listdir('../lib'))

# 2. what does lib *actually* export?
import lib
print('lib dir:', [x for x in dir(lib) if not x.startswith('_')])

In [2]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from lib import Sequential, Dense, Tanh, Sigmoid, MSELoss, SGD

model = Sequential([
    Dense(2,4), Tanh(),
    Dense(4,1), Sigmoid()
])
opt = SGD(lr=1.0)

### Training loop

In [3]:
losses=[]
for epoch in range(5000):
    loss = model.train_step(X, y)
    opt.step(model.layers)
    losses.append(loss)
    if epoch % 500 == 0:
        print(epoch, loss)

0 0.25783236285340533
500 0.0016000043578739934
1000 0.0006749917114371051
1500 0.00041579788146404756
2000 0.00029651483569238845
2500 0.00022866939207225905
3000 0.00018517265001321377
3500 0.00015504158002280674
4000 0.00013300441220111915
4500 0.00011622505875615656


### Assert correctness

In [4]:
pred = model.forward(X)
print("raw preds:\n", pred)
print("rounded:\n", np.round(pred))

raw preds:
 [[0.00572589]
 [0.98904277]
 [0.9894438 ]
 [0.01216161]]
rounded:
 [[0.]
 [1.]
 [1.]
 [0.]]


### Gradient-check block

In [5]:
# from lib.layers import Dense
# from lib.losses import MSELoss
layer = Dense(2,4);  _ = layer.forward(X)
loss_fn = MSELoss(); eps = 1e-5
numeric = np.zeros_like(layer.W)
for i in range(layer.W.shape[0]):
    for j in range(layer.W.shape[1]):
        layer.W[i,j] += eps
        plus  = loss_fn(y, layer.forward(X))
        layer.W[i,j] -= 2*eps
        minus = loss_fn(y, layer.forward(X))
        layer.W[i,j] += eps
        numeric[i,j] = (plus - minus)/(2*eps)
# compare with analytical
y_pred = layer.forward(X)
dout = loss_fn.backward()
_, analytic, _ = layer.backward(dout)
print('Max diff:', np.max(np.abs(analytic - numeric)))  # <1e-7 → pass

Max diff: 1.69538170861696
